In [0]:
!pip install deeppavlov 

In [0]:
!pip install tensorflow==1.15.0 

In [0]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu 

In [0]:
!wget https://raw.githubusercontent.com/PetrFil/ComplingHSE/master/test.txt
!wget https://raw.githubusercontent.com/PetrFil/ComplingHSE/master/train.txt
!wget https://raw.githubusercontent.com/PetrFil/ComplingHSE/master/valid.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True) 

In [0]:
marked = []

for text in data.text.values[:1000]:
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [0]:
marked

1) Любопытно, что в моем файле Valid.txt "Звезды Playstation Битва сильнейших" размечена как игра (по крайней мере так называется сборник игр). Обученная же модель демонстрирует такой результат: ('Звезды', 'O'), ('Playstation', 'O'),('Битва', 'O'), ('сильнейших', 'O'). По всей видимости, потому что слово Playstation всё же чаще размечалось как "О".
2) Игра "Дурная репутация 2" стоимостью 600 р. не определилась как именованная сущность. Вероятно, слишком мало кириллических названий в обучающей выборке. Потдверждают эту гипотезу и разметка соседствующих "Far Cry 4" и "фар край 4". Первая из них выделилась как именованная сущность, а вторая нет.
3) Объявление из 3 слов: "Джойстик и провода". Почему-то "Джойстик" обозначен как игра. Некоторые короткие объявления сразу начинаются с названий игр. Возможно также играет роль и нулевая частотность этого слова в обучающей выборке. Однако с "Дурной репутацией 2" так не произошло
4) Одно из объявлений обучающей выборки состояло из внушительного списка игр, написанного полностью в верхнем регистре. Поэтому обученная модель решила, что предложение "ПРОДАЮ НОВЫЙ ДИСК ТОРГ УМЕСТЕН ИЛИ ОБМЕН" это название одной игры. 
5) Возможно модель считает, что некоторые цифры более характерны для игр (из более очевидного - 2, 3, 4), а другие нет ( 600, 1200, 1500 - это обычно цены, например). В следующем списке "Murdered 300, Infamous 500, Assassin's creed 800, The division 1200"число "500" было отмечено моделью как продолжение именованной сущности, хотя остальные цены в данном контексте были верно распознаны как что-то отличное от именованной сущности. 


In [27]:
ner_model(['Devil May Cry', 'Продам настольную игру Шахматы', 'Джойстик', 'СОВЕРШЕННО СЛУЧАЙНЫЙ ТЕКСТ, НАБРАННЫЙ С КАПС ЛОКОМ', 'ps4 - это не игра, а приставка'])

[[['Devil', 'May', 'Cry'],
  ['Продам', 'настольную', 'игру', 'Шахматы'],
  ['Джойстик'],
  ['СОВЕРШЕННО', 'СЛУЧАЙНЫЙ', 'ТЕКСТ', ',', 'НАБРАННЫЙ', 'С', 'КАПС', 'ЛОКОМ'],
  ['ps4', '-', 'это', 'не', 'игра', ',', 'а', 'приставка']],
 [['B-Game', 'I-Game', 'I-Game'],
  ['O', 'O', 'O', 'O'],
  ['B-Game'],
  ['O', 'B-Game', 'I-Game', 'O', 'B-Game', 'I-Game', 'I-Game', 'I-Game'],
  ['B-Game', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]]

Некоторые другие найденные ошибки без комментариев:

"Thief's", 'I-Game'),
  ('End', 'I-Game'),
  ('(', 'I-Game'),
  ('PS4', 'O'),
  (')', 'O'),


  ('xbox', 'B-Game'), ('one', 'I-Game')
  'ps3', 'B-Game'
    ('box', 'O'), ('360', 'O')

  ('Toefl', 'B-Game'),
  ('Test', 'I-Game'),
  ('-', 'O'),
  ('Рекордный', 'O'),
  ('Результат', 'O'),
  ('-', 'O'),
  ('50', 'O'),
  ('рублей', 'O'),

  'Nyko', 'B-Game'

  ('Средиземье', 'O'),
  (':', 'O'),
  ('Тени', 'B-Game'),
  ('Мордора', 'I-Game')

  'The', 'B-Game'),
  ('Ship', 'I-Game'),
  ('Остаться', 'O'),
  ('в', 'O'),
  ('живых', 'O')


  ('ПРОДАЮ', 'B-Game'),
  ('НОВЫЙ', 'I-Game'),
  ('ДИСК', 'I-Game'),
  ('ТОРГ', 'I-Game'),
  ('УМЕСТЕН', 'I-Game'),
  ('ИЛИ', 'I-Game'),
  ('ОБМЕН', 'I-Game')],

  'слифин', 'B-Game'),
  ('догс', 'I-Game'),
  ('вачь', 'I-Game'),
  ('догс', 'I-Game'),
  ('гат', 'I-Game'),
  ('5', 'I-Game'),

  'Murdered', 'B-Game'),
  ('300', 'O')

  ('Infamous', 'B-Game'),
  ('500', 'I-Game'),

  "Assassin's", 'B-Game'),
  ('creed', 'I-Game'),
  ('800', 'O'),

  ('The', 'B-Game'),
  ('division', 'I-Game'),
  ('1200', 'O'),

  ('Assassins', 'B-Game'), ('Creed', 'I-Game'), ('Единство', 'O')]

  'Uncharted', 'B-Game'),
  ('4', 'I-Game'),
  (':', 'O'),
  ('Путь', 'O'),
  ('вора', 'O'),

  'компьютерная', 'O'),
  ('игра', 'O'),
  ('\xa0', 'O'),
  ('в', 'O'),
  ('жанре', 'O'),
  ('\xa0', 'O'),
  ('action', 'B-Game'),
  ('-', 'O'),

  SonyPlayStation', 'B-Game'),
  ('4', 'I-Game'),
   [('Uncharted', 'B-Game'),
  ('бездна', 'O'),
  ('рейман', 'O'),
  ('1и', 'O'),
  ('2', 'O'),
  ('резистенс', 'O'),
  ('нинза', 'O'),
  ('гайден', 'O'),
  ('2', 'O')


  ('Far', 'B-Game'),
  ('cry', 'I-Game'),
  ('4', 'I-Game'),
  (',', 'O'),
  ('фар', 'O'),
  ('край', 'O'),
  ('4', 'O'),

  'Epic', 'B-Game'),
  ('Mickey', 'I-Game'),
  ('Две', 'O'),
  ('легенды', 'O')